In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS fleetops_project.gold;

In [0]:
%sql
CREATE OR REPLACE TABLE fleetops_project.gold.dim_caminhao AS
SELECT 
    truck_id AS caminhao_sk,
    plate AS placa,
    model AS modelo,
    capacity_kg AS capacidade,
    manufacture_year AS ano_fabricacao
FROM fleetops_project.silver.trucks;

In [0]:
%sql
CREATE OR REPLACE TABLE fleetops_project.gold.dim_motorista AS
SELECT 
    driver_id AS motorista_sk,
    name AS nome,
    cnh,
    cnh_type AS tipo_cnh
FROM fleetops_project.silver.drivers;

In [0]:
%sql

CREATE OR REPLACE TABLE fleetops_project.gold.fact_telemetria AS
SELECT 
    t.event_id,
    t.event_time AS data_hora,
    t.truck_id,
    
    COALESCE(da.assignment_sk, -1) AS alocacao_sk,
    COALESCE(da.driver_id, -1) AS motorista_fk,
    
    t.speed_kmh AS velocidade,
    t.engine_rpm AS rpm,
    t.fuel_level AS nivel_combustivel,
    t.engine_temp AS temperatura_motor,
    t.gps_latitude,
    t.gps_longitude,
    
    CASE WHEN t.speed_kmh > 100 THEN 1 ELSE 0 END AS flag_excesso_velocidade,
    CASE WHEN t.engine_rpm > 3500 THEN 1 ELSE 0 END AS flag_rotacao_alta,
    CASE WHEN t.engine_temp > 105 THEN 1 ELSE 0 END AS flag_superaquecimento

FROM fleetops_project.silver.iot_telemetry t
LEFT JOIN fleetops_project.silver.driver_assignments_scd da
    ON t.truck_id = da.truck_id
    AND t.event_time >= da.effective_start
    AND t.event_time < da.effective_end;
    
OPTIMIZE fleetops_project.gold.fact_telemetria ZORDER BY (truck_id, data_hora);

In [0]:
%sql
CREATE OR REPLACE TABLE fleetops_project.gold.agg_driver_performance_daily AS
SELECT
    f.motorista_fk,
    m.nome as nome_motorista,
    DATE(f.data_hora) as data_referencia,
    
    COUNT(CASE WHEN f.flag_excesso_velocidade = 1 THEN 1 END) as qtd_infracoes_velocidade,
    COUNT(CASE WHEN f.flag_rotacao_alta = 1 THEN 1 END) as qtd_abuso_motor,
    COUNT(CASE WHEN f.flag_superaquecimento = 1 THEN 1 END) as qtd_superaquecimento,
    
    COUNT(f.event_id) as tempo_operacao_total_minutos,
    COUNT(CASE WHEN f.velocidade = 0 AND f.rpm > 0 THEN 1 END) as tempo_ocioso_minutos,
    (MAX(f.nivel_combustivel) - MIN(f.nivel_combustivel)) as consumo_tanque_percentual,
    
    AVG(f.rpm) as rpm_medio_diario,
    MAX(f.velocidade) as velocidade_maxima_atingida,
    
    GREATEST(0, 100 - (
        (COUNT(CASE WHEN f.flag_excesso_velocidade = 1 THEN 1 END) * 5) +
        (COUNT(CASE WHEN f.flag_rotacao_alta = 1 THEN 1 END) * 2) +
        (COUNT(CASE WHEN f.flag_superaquecimento = 1 THEN 1 END) * 10)
    )) AS score_conducao

FROM fleetops_project.gold.fact_telemetria f
LEFT JOIN fleetops_project.gold.dim_motorista m 
    ON f.motorista_fk = m.motorista_sk
WHERE f.motorista_fk <> -1
GROUP BY 1, 2, 3;